# Image Classification

In [1]:
import os
import matplotlib.pyplot as olt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms,models,datasets
import imageio
import time
import warnings
import random
import sys
import copy
import json
from PIL import Image

### Import Training and Validation Image

In [2]:
data_dir='./flower_data/'
train_dir=data_dir+'/train'
valid_dir=data_dir+'/valid'

### Data Pre-Processsing

In [3]:
# Data Augmentation
data_transforms={
    'train':transforms.Compose([transforms.RandomRotation(45),                  # Random Rotation between -45 to +45 degree
    transforms.CenterCrop(224),                                                 # Cut from Center, VGG, Resnet
    transforms.RandomHorizontalFlip(p=0.5),                                     # Random Horizontal Flip
    transforms.RandomVerticalFlip(p=0.5),                                       # Random Vertical Flip
    transforms.ColorJitter(brightness=0.2,contrast=0.1,saturation=0.1,hue=0.1), # brightness=0.2,contrast=0.1,saturation=0.1,hue=0.1
    transforms.RandomGrayscale(p=0.025),                                        # Probability to get Grayscale
    transforms.ToTensor(),
    transforms.Normalize([0.458,0.456,0.406],[0.229,0.224,0.225])               # Mean and Var, make sure our data fit other model like VGG Resnet
    ]),
    'valid':transforms.Compose([transforms.Resize(256),
    transforms.CenterCrop(224),                                                 # Cut from Center
    transforms.ToTensor(),
    transforms.Normalize([0.458,0.456,0.406],[0.229,0.224,0.225])               # Mean and Var
    ])
}

In [7]:
batch_size=8
image_datasets={x:datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x]) for x in ['train', 'valid']} # root=os.path.join(data_dir,x), transforms=data_transforms[x], in ['train', 'valid']
dataloaders={x:torch.utils.data.DataLoader(image_datasets[x],batch_size=batch_size,shuffle=True) for x in ['train', 'valid']}
dataset_sizes={x:len(image_datasets[x]) for x in ['train', 'valid']}
class_name=image_datasets['train'].classes

with open ('cat_to_name.json','r') as f: # Read name of flower
    cat_to_name=json.load(f)


In [9]:
def im_convert(tensor):
    '''Convert tensor data type back to noraml image data type'''
    image = tensor.to('cpu').clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1,2,0)
    image = image * np.array((0.229,0.224,0.225))+np.array((0.458,0.456,0.406))
    image = image.clip(0,1)
    return image